In [1]:
from utils import *

hete = Heterogeneity()
hete.check_torch_gpu()

-------------------------------------------------
------------------ VERSION INFO -----------------
Conda Environment: torchy | Python version: 3.8.16 (default, Mar  2 2023, 03:18:16) [MSC v.1916 64 bit (AMD64)]
Torch version: 2.0.1
Torch build with CUDA? True
# Device(s) available: 1, Name(s): Quadro P520



***
# END

In [ ]:
X_data = np.load('X_data.npy')
y_data = np.load('y_data.npy')
print(X_data.shape, y_data.shape)

In [ ]:
all_data = np.concatenate([X_data, y_data], -1)
print(all_data.shape)

In [ ]:
X_data[0,:,:,0, 2].max()

In [ ]:
realization = 2
fig, axs = plt.subplots(6, 12, figsize=(20,10))
for i in range(6):
    for j in range(12):
        axs[i,j].imshow(all_data[realization, :, :, j*5, i], 'jet')
        axs[i,j].set(xticks=[], yticks=[])
        if i==3:
            im = axs[i,j].imshow(all_data[realization,:,:,j*5,i], 'jet', vmin=0, vmax=60)
            plt.colorbar(im, pad=0.04, fraction=0.046)
plt.show()

In [ ]:
device = 'cuda'

rom = h2_hete_rom().to(device)
optimizer = NAdam(rom.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

X_train = torch.Tensor(X_data).to(device)
y_train = torch.Tensor(y_data).to(device)

loss, val_loss = [], []
metrics = {'loss':[], 'val_loss':[]}
epochs = 10
batch_size = 1

for epoch in range(epochs):
    rom.train()
    epoch_loss = 0.0
    for i in range(0, len(X_train), batch_size):
        inp  = X_train[i:i+batch_size]
        true = y_train[i:i+batch_size]
        optimizer.zero_grad()
        pred = rom(inp)
        loss = loss_fn(pred,true)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()*inp.size(0)
    metrics['loss'].append(epoch_loss/len(X_train))